In [3]:
from urllib2 import urlopen
from bs4 import BeautifulSoup
import csv
import re

In [4]:
# URL that contains partial view of and link to each door
# obtained manually
panelDoors_links = ["http://www.trustile.com/products/panel-doors/ts-series",
                  "http://www.trustile.com/products/panel-doors/tm-series",
                  "http://www.trustile.com/products/panel-doors/vg-series",
                  "http://www.trustile.com/products/panel-doors/tmb-series",
                  "http://www.trustile.com/products/panel-doors/cd-series"]
glassDoors_links = ["http://www.trustile.com/products/glass-doors/fl-series",
                  "http://www.trustile.com/products/glass-doors/pl-series"]
otherDoors_links = ["http://www.trustile.com/products/lvr-series",
                  "http://www.trustile.com/products/exterior-doors",
                  "http://www.trustile.com/products/fire-rated-doors",
                  "http://www.trustile.com/products/custom-doors"]

In [5]:
panelDoors = []
for p in range(len(panelDoors_links)):
    # get webpage of each door series
    # and make structured data using BeautifulSoup
    link = panelDoors_links[p]
    soup = BeautifulSoup(urlopen(link).read())
    
    aList = []
    for url in soup.findAll('a'):
        try:
            urlClass = url.get('class')
            if urlClass[0]=="gridview":
                aList.append(url.get('href'))
#             print url.get('href')
            
        except:
            pass
    panelDoors.append(aList)

In [6]:
glassDoors = []
for g in range(len(glassDoors_links)):
    # get webpage of each door series
    # and make structured data using BeautifulSoup
    link = glassDoors_links[g]
    soup = BeautifulSoup(urlopen(link).read())
    
    aList = []
    for url in soup.findAll('a'):
        try:
            urlClass = url.get('class')
            if urlClass[0]=="gridview":
                aList.append(url.get('href'))
            
        except:
            pass
    glassDoors.append(aList)
print len(glassDoors[0]), len(glassDoors[1])

99 49


In [7]:
otherDoors = []
for o in range(len(otherDoors_links)):
    # get webpage of each door series
    # and make structured data using BeautifulSoup
    link = otherDoors_links[o]
    soup = BeautifulSoup(urlopen(link).read())
    
    aList = []
    for url in soup.findAll('a'):
        try:
            urlClass = url.get('class')
            if urlClass[0]=="gridview":
                aList.append(url.get('href'))
            
        except:
            pass
    otherDoors.append(aList)

for o in range(len(otherDoors_links)):
    print len(otherDoors[o])

40
0
0
0


In [8]:
base_url = "http://www.trustile.com/"

In [10]:
def make_url(str):
    return base_url + str

In [11]:
def make_soup(url):
    html = urlopen(url).read()
    return BeautifulSoup(html)

In [12]:
def get_doorName(soup):
    return soup.h1.string

In [13]:
def get_pageFields(soup):
    return soup.findAll('fieldset')

In [16]:
def get_images(field):
    # get all types of images of this door model
    div = field.find('div',attrs={"class":"doorthumbs"})
    images = []
    for a in div.findAll('a'):
        aImage = {}
        aImage["type"] = a.get('alt')
        aImage["url"] = a.get('href')
        images.append(aImage)
    return images
            

In [59]:
# detail information on the right of the page
def get_details(field):
    div = field.find('div',attrs={"class":"doordetails"})
    details = {}
    for p in div.findAll('p'):
        try:
            ifDetail = p.get('class')[2]
            if ifDetail == "product-detail":           
                category = p.span
                if category.string == u'Series:\xa0':
                    details['Series'] = p.a.string
    
                if category.string == u'Application:\xa0':
                    stri = p.get_text()
                    app = re.search(u'(?<=\xa0).*',stri).group()
                    details['Application'] = app
    
                if category.string == u'Thickness:\xa0':
                    details['Thickness'] = p.findAll('span')[1].get_text()
    
                if category.string == u'Construction:\xa0':
                    construction = []
                    for child in p.findAll('span')[1].children:
                        construction.append(child.string)
                    details['Construction'] = construction
                if category.string == u'Available Fire Rating:\xa0':
                    fireRating = []
                    for img in p.findAll('span')[1].findAll('img'):
                        aRating = {}
                        aRating["type"] = img.get('alt')
                        aRating["url"] = img.get('src')
                        fireRating.append(aRating)
                    details['Fire'] = fireRating
        except:
            print p.get_text()
    return details

In [61]:

def get_profiles(field):
    for view in field.findAll('fieldset'):
        # get option names
        if view.legend.span.string == "Preview Profiles":
            # sticking options
            sticking = view.find("div",attrs={"id":"sticking"})
            stickingOptions = []
            for p in sticking.findAll("p"):
                ab = re.search(u'.*(?=\u2009\u2013)',p.label.string).group()
                full = re.search(u'(?<=\u2013\u2009).*',p.label.string).group()
#                 aOption = {}
#                 aOption["type"] = ab
#                 aOption["name"] = full
#                 stickingOptions.append(aOption)
                stickingOptions.append(ab)
                
            # panel options
            panels = view.find("div",attrs={"id":"panels"})
            panelOptions = []
            for p in panels.findAll("p"):
                ab = re.search(u'.*(?=\u2009\u2013)',p.label.string).group()
                full = re.search(u'(?<=\u2013\u2009).*',p.label.string).group()
#                 aOption = {}
#                 aOption["type"] = ab
#                 aOption["name"] = full
#                 panelOptions.append(aOption)
                panelOptions.append(ab)
            
        # add option images
#         if view.legend.span.string == "View Profile Cross-Sections":
#             # panel options
#             table1 = view.findAll('table')[0]
#             for td in table1.findAll('td'):
#                 imageURL =  td.a.get('href')
#                 label = td.span.get_text()
#                 key = re.search(u'(?<=\u201c).(?=\u201d)',label).group()
#                 for option in panelOptions:
#                     if option["type"] == key:
#                         option["url"] = imageURL
                
#             # sticking options
#             table2 = view.findAll('table')[1]
#             for td in table2.findAll('td'):
#                 try:
#                     imageURL = td.a.get('href')
#                     label = td.span.get_text()
#                     key = re.search(u'(?<=\().*(?=\)\s)',label).group()
#                     for option in stickingOptions:
#                         if option["type"] == key:
#                             option["url"] = imageURL
#                 except:
#                     pass
    return stickingOptions, panelOptions

In [29]:
# get information for tab Wood, Glass, Leather, Metal
def get_fromGridTab(field):
    grid = []
    for li in field.findAll('li'):
        cell = dict([("type",li.span.string),("url",li.a.get('href'))])
        grid.append(cell)
    return grid

###Scrape name, images and details on Overview for each door model

In [62]:
# 
# TS series
aList = panelDoors[0]
outFile = open("TrustilePanelDoors_TS.csv", "wb")
writer = csv.writer(outFile)
writer.writerow(["","Name","Images","Series","Application","Construction","Thickness","Fire Rating","Sticking","Panel","Options"])

for i in range(len(aList)):
    soup = make_soup(make_url(aList[i]))
    
    name = get_doorName(soup)
    
    options = []
    fields = get_pageFields(soup)
    for field in fields:
        if field.legend.span.string == "Overview":
            images = get_images(field)
            
            details = get_details(field)
            try:
                fireRating = details["Fire"]
            except:
                fireRating = ""
        
        if field.legend.span.string == "Profiles":
            sticking, panel = get_profiles(field)
        
        
        # tab3: wood species
        if field.legend.span.string == "Wood Species":
#             woodSpecies = get_fromGridTab(field)
            options.append('Wood')
    
        # tab4: glass options
        if field.legend.span.string == "Glass":
#             glassOptions = get_fromGridTab(field)
            options.append('Glass')
        
        # tab5: leather options
        if field.legend.span.string == "Leather":
#             leatherOptions = get_fromGridTab(field)
            options.append('Leather')
    
        # tab6: Metal options
        if field.legend.span.string == "Metal":
#             metalOpions = get_fromGridTab(field)
            options.append('Metal')
            
    writer.writerow([i, name, images, 
                     details["Series"], details["Application"], details["Construction"], details["Thickness"], fireRating,
                     sticking, panel,
                     options])
outFile.close()

In [41]:
sticking = open("TrustileTSsticking.csv","wb")
w = csv.writer(sticking)
w.writerow(["Type","Name","URL"])
for s in S:
    w.writerow([s["type"],s["name"],s["url"]])
sticking.close()

In [42]:
panel = open("TrustileTSpanel.csv","wb")
w = csv.writer(panel)
w.writerow(["Type","Name","URL"])
for p in P:
    w.writerow([p["type"],p["name"],p["url"]])
panel.close()